In [ ]:
import re
from collections import defaultdict

## Step 1: Load and Clean the Text
def clean_text(text):
    # Remove non-alphabetic characters. Keep letters, spaces and full stops
    cleaned = re.sub(r'[^A-Z\s.]','', text.upper())
    return cleaned

# Function to load in Text files and clean them
def process_file(file_path):
    with open(file_path, 'r', encoding='etf-8') as f:
        text = f.read()

    # Remove preamble/postamble for the texts from Project Guttenberg
    text = text.split("*** START OF THIS PROJECT GUTENBERG BOOK" )[1]
    text = text.split("*** END OF THIS PROJECT GUTENBERG BOOK" )[0]

    # Clean the text
    return clean_text(text)

# Example with one file
cleaned_text = process_file('GutenberTexts/Frankenstein.txt')
# Display the first 500 characters of Frankenstein
print(cleaned_text[:500])  

def build_trigram_model(cleaned_text):
    trigram_counts = defaultdict(int)

    # Loop through the text and the create trigrams
    for i in range(len(cleaned_text) -2):
        trigram = cleaned_text[i:i+3]
        trigram_counts[trigram] += 1

    return trigram_counts

